In [121]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from imutils import paths
import random
import time

In [122]:
import cv2
from skimage.feature import hog
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import GridSearchCV
import json

In [123]:
annotations = json.load(open('images/Annotations.json'))
class_object = list(annotations['labels'])

In [124]:
train_images = []
train_labels = []

In [125]:
def load_images(image_paths):
    for (i, image_path) in enumerate(image_paths):
        image = cv2.imread(image_path)
        image = np.array(image).astype('uint8')
        image = cv2.resize(image, (64, 64))
        grey_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        hog_features, hog_images = hog(grey_image,
                              visualize=True,
                              block_norm='L2-Hys',
                              pixels_per_cell=(16, 16))
        #label image using the annotations
        label = class_object.index(annotations["annotations"][image_path[7:]][0]['label'])
        train_images.append(hog_features)
        train_labels.append(label)

In [126]:
image_paths = list(paths.list_images('images'))
load_images(image_paths)

In [127]:
train_array = np.array(train_images)
train_array = np.vstack(train_array)
labels_array = np.array(train_labels)

labels_array = labels_array.astype(int)
labels_array = labels_array.reshape((labels_array.size,1))
train_df = np.concatenate([train_array, labels_array], axis = 1)

In [128]:
y = train_df[:, -1:].reshape(-1,)
y.shape

(200,)

In [129]:
x_train, x_test, y_train, y_test = train_test_split(train_df[:, :-1], y, test_size= 0.2, random_state= 365)

In [130]:
param_grid = {'kernel': ['rbf'],'C': [10]}

In [131]:
base_estimator = SVC(gamma='scale')

In [132]:
from datetime import datetime

start_datetime = datetime.now()
start = time.time()

svm = GridSearchCV(base_estimator, param_grid, cv=4)
#Fit the data into the classifier
svm.fit(x_train,y_train)
#Get values of the grid search
best_parameters = svm.best_params_
print(best_parameters)
#Predict on the validation set
y_pred = svm.predict(x_test)
# Print accuracy score for the model on validation  set.
print("Accuracy: "+str(accuracy_score(y_test, y_pred)))

end = time.time()
end_datetime = datetime.now()
print(end - start)

{'C': 10, 'kernel': 'rbf'}
Accuracy: 0.675
0.04500412940979004


In [133]:
label_names = [0, 1]
cmx = confusion_matrix(y_test, y_pred, labels=label_names)
cmx

array([[12,  4],
       [ 9, 15]])